In [1]:
import tensorflow_datasets as tfds 


2022-07-10 13:30:29.994621: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-10 13:30:29.994658: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [35]:
(kmnist_tr, kmnist_ts) = tfds.load('kmnist', 
                                   split=['train','test'],
                                   shuffle_files=True,
                                   as_supervised=True,
                                   with_info=False,)


In [3]:
(emnist_tr, emnist_ts) = tfds.load('emnist', 
                                   split=['train','test'],
                                   shuffle_files=True,
                                   as_supervised=True,
                                   with_info=False,)


2022-07-10 13:31:40.867111: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

Generating train examples...:   0%|          | 0/697932 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/emnist/byclass/3.0.0.incomplete80YBUX/emnist-train.tfrecord*...:   0%|        …

Generating test examples...:   0%|          | 0/116323 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/emnist/byclass/3.0.0.incomplete80YBUX/emnist-test.tfrecord*...:   0%|         …

Dataset emnist downloaded and prepared to ~/tensorflow_datasets/emnist/byclass/3.0.0. Subsequent calls will reuse this data.


In [20]:
liste_kmnist_tr = list(kmnist_tr)
liste_emnist = list(emnist_tr)

In [16]:
liste_kmnist_tr[0][0].shape, liste_kmnist_tr[0][1].shape

(TensorShape([28, 28, 1]), TensorShape([]))

In [39]:
y_liste_kmnist_tr = [y for _, y in liste_kmnist_tr]
#y_liste_emnist = [y for _, y in liste_emnist]

In [40]:
min(y_liste_kmnist_tr), max(y_liste_kmnist_tr), min(y_liste_emnist), max(y_liste_emnist)

(<tf.Tensor: shape=(), dtype=int64, numpy=60>,
 <tf.Tensor: shape=(), dtype=int64, numpy=69>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=61>)

In [41]:
kmist_tr = kmnist_tr.apply(lambda ds: ds.map(lambda x, y: (x, y + 60)))
kmnist_ts = kmnist_ts.apply(lambda ds: ds.map(lambda x, y: (x, y + 60)))

In [38]:
liste_kmnist_tr[0][1]

<tf.Tensor: shape=(), dtype=int64, numpy=69>

In [42]:
kmist_tr = kmist_tr.concatenate(emnist_tr)
kmnist_ts = kmnist_ts.concatenate(emnist_ts)

<ConcatenateDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [89]:
import numpy as np 
import tensorflow as tf # use the tf2.4
import tensorflow_datasets as tfds 
import tensorflow_addons as tfa
from adabelief_tf import AdaBeliefOptimizer
inner_lr = 1E-5
outter_lr = 1E-5
inner_task_loop_no = 64 # give 32 tasks of 2ways-1shot
# opt_inner = tf.keras.optimizers.Adagrad(inner_lr)
# opt_outter = tf.keras.optimizers.Adagrad(outter_lr, clipnorm=1.)
opt_inner = tfa.optimizers.NovoGrad(inner_lr)
opt_outter = tfa.optimizers.NovoGrad(outter_lr, clipnorm=1.)
# opt_inner = tfa.optimizers.RectifiedAdam(inner_lr)
# opt_outter = tfa.optimizers.RectifiedAdam(outter_lr, clipnorm=1.)
# opt_inner = AdaBeliefOptimizer(inner_lr)
# opt_outter = AdaBeliefOptimizer(outter_lr)
# opt_inner = tf.keras.optimizers.SGD(inner_lr, momentum=.0)
# opt_outter = tf.keras.optimizers.SGD(outter_lr, momentum=.9, clipnorm=1.)

#opt_outter = tfa.optimizers.MovingAverage(opt_outter) # using average strategy in tfa

def cnn():
    Input = tf.keras.Input([28, 28, 1])
    Input_n = Input/128.0 - 1
    conv1 = tf.keras.layers.Conv2D(32, [3, 3], strides=(2, 2), padding="SAME", kernel_initializer="he_uniform", activation=tf.nn.relu)(Input_n) #[14,14]
    conv2 = tf.keras.layers.Conv2D(64, [3, 3], strides=(2, 2), padding="SAME", kernel_initializer="he_uniform", activation=tf.nn.relu)(conv1) #[7,7]
    conv3 = tf.keras.layers.Conv2D(128, [3, 3], strides=(2, 2), padding="SAME", kernel_initializer="he_uniform", activation=tf.nn.relu)(conv2) #[4,4]
    fc = tf.keras.layers.Flatten()(conv3)
    fc1 = tf.keras.layers.Dense(128, kernel_initializer="he_uniform", activation=tf.nn.relu)(fc)
    fc2 = tf.keras.layers.Dense(256, kernel_initializer="he_uniform", activation=tf.nn.relu)(fc1)
    fc3 = tf.keras.layers.Dense(512, kernel_initializer="he_uniform", activation=tf.nn.relu)(fc2)
    out = tf.keras.layers.Dense(1, kernel_initializer="he_uniform", activation=None)(fc3)
    return tf.keras.Model(inputs=Input, outputs=out)
pass 

def select_support_query_set(dataset_iter):
    flag_1 = flag_0 = True
    imgs_1_support, labs_1_support = next(dataset_iter)
    imgs_0_support, labs_0_support = next(dataset_iter)
    support_set = tf.concat([imgs_1_support, imgs_0_support], axis=0)
    while(flag_1 or flag_0):
        imgs_query, labs_query = next(dataset_iter)
        if ((labs_query.numpy() == labs_1_support.numpy()) and flag_1) :
            imgs_1_query = tf.Variable(imgs_query)
            flag_1 = False
        elif ((labs_query.numpy() == labs_0_support.numpy()) and flag_0) :
            imgs_0_query = tf.Variable(imgs_query)
            flag_0 = False
        pass 
    pass 
    query_set = tf.concat([imgs_1_query, imgs_0_query], axis=0)
    return [support_set, query_set]
pass 

(kmnist_tr, kmnist_ts) = tfds.load('kmnist', 
                                   split=['train','test'],
                                   shuffle_files=True,
                                   as_supervised=True,
                                   with_info=False,)

kmist_tr = kmnist_tr.apply(lambda ds: ds.map(lambda x, y: (x, y + 60)))
kmnist_ts = kmnist_ts.apply(lambda ds: ds.map(lambda x, y: (x, y + 60)))

(emnist_tr, emnist_ts) = tfds.load('emnist', 
                                   split=['train','test'],
                                   shuffle_files=True,
                                   as_supervised=True,
                                   with_info=False,)

kmist_tr = kmist_tr.concatenate(emnist_tr)
kmnist_ts = kmnist_ts.concatenate(emnist_ts)



# Task hint:
# this example will use 2ways-1shot for training the MAML.
# the inner task loop will set as 30. 
kmnist_tr = kmnist_tr.batch(1).repeat()
kmnist_tr_iter = iter(kmnist_tr)
meta_labs = tf.Variable([[1], [0]], dtype=tf.float32)

cnn_model = cnn()

# according to the TF2 manual, the variable of the model would need to be
# initialized, and use it to inferece something is a way to initialize 
# the varialbe.
support_set, query_set = select_support_query_set(kmnist_tr_iter)
_ = cnn_model(support_set) # 1)for initializing the model. 2)for define the loss function out-of the loop 
loss_inner_meta = lambda: tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=meta_labs, logits=cnn_model(support_set)))
loss_inner_task = lambda: tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=meta_labs, logits=cnn_model(query_set)))

for step in range(50000):
    # keep the original weights for meta-training
    meta_weights = [tf.Variable(target_weights) for target_weights in cnn_model.trainable_weights]
    
    def meta_loss():
        loss_outter = 0
        for task_count in range(inner_task_loop_no):
            # fetch the dataset. The first dataset is always be 1. 
            # So, the labels will be defined as the meta-labels insteadly. 
            support_set, query_set = select_support_query_set(kmnist_tr_iter)
            
            # put the meta-weights into the model
            for model_weight_index in range(len(cnn_model.trainable_weights)):
                cnn_model.trainable_weights[model_weight_index].assign(meta_weights[model_weight_index])
            pass

            # updateing the weights from meta-weights
            opt_inner.minimize(loss=loss_inner_meta, var_list=cnn_model.trainable_weights) 
            
            # calculating the task loss (after the meta-weights updating)
            # here, the same support set and query set are used. you can 
            # also creat a different query set for meta-weight training.
            loss_outter += loss_inner_task()
        pass 
        return loss_outter/inner_task_loop_no
    pass 

    # put the meta-weights into the model
    for model_weight_index in range(len(cnn_model.trainable_weights)):
        cnn_model.trainable_weights[model_weight_index].assign(meta_weights[model_weight_index])
    pass

    # update the meta-weights
    opt_outter.minimize(loss=meta_loss, var_list=cnn_model.trainable_weights) 

    print("step: {}  outter_loss: {}".format(step, meta_loss()))
pass 


step: 0  outter_loss: 0.6384350657463074
step: 1  outter_loss: 0.6382899880409241
step: 2  outter_loss: 0.6381451487541199
step: 3  outter_loss: 0.6380004286766052
step: 4  outter_loss: 0.6378491520881653
step: 5  outter_loss: 0.637701690196991
step: 6  outter_loss: 0.6375542283058167
step: 7  outter_loss: 0.6374064087867737
step: 8  outter_loss: 0.6372604966163635
step: 9  outter_loss: 0.6371151804924011
step: 10  outter_loss: 0.6369689702987671
step: 11  outter_loss: 0.6368231177330017
step: 12  outter_loss: 0.6366778612136841
step: 13  outter_loss: 0.636538565158844
step: 14  outter_loss: 0.636386513710022
step: 15  outter_loss: 0.6362319588661194
step: 16  outter_loss: 0.6360780000686646
step: 17  outter_loss: 0.6359229683876038
step: 18  outter_loss: 0.6357686519622803
step: 19  outter_loss: 0.6356228590011597
step: 20  outter_loss: 0.6354789137840271
step: 21  outter_loss: 0.6353362798690796
step: 22  outter_loss: 0.6352033615112305
step: 23  outter_loss: 0.635063111782074
step: 

KeyboardInterrupt: 

In [90]:
cnn_model(tf.zeros([2, 28, 28]))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.03123168],
       [0.03123135]], dtype=float32)>

In [22]:
cnn_model(support_set)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.87742525],
       [-1.1806308 ]], dtype=float32)>

In [91]:
wünscheswertes_char = 1
potenzielle_chars = list()
nicht_potenzielle_chars = list()
for x, y in kmnist_tr:
    if y == wünscheswertes_char:
        potenzielle_chars.append((x, y))
    else:
        nicht_potenzielle_chars.append([x, y])
        
    if len(potenzielle_chars) > 20 and len(nicht_potenzielle_chars) > 20:
        break


In [35]:
len(potenzielle_chars)

21

In [79]:
[y for x, y in potenzielle_chars]

[<tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=int64, numpy=array([1])>,
 <tf.Tensor: shape=(1,), dtype=

In [78]:
clone_cnn_model = tf.keras.models.clone_model(cnn_model)
clone_cnn_model.compile( tf.keras.optimizers.SGD() , tf.keras.losses.BinaryCrossentropy(from_logits=True))

for x, y in potenzielle_chars:
    clone_cnn_model.fit(x, tf.ones([1, 1]))
for x, y in nicht_potenzielle_chars:
    clone_cnn_model.fit(x, tf.zeros([1, 1]))

1/1 [==============================] - 0s 5ms/step - loss: 2.7122e-04


In [92]:
stimt = list()
i = 0
for x, y in potenzielle_chars:
    pred = clone_cnn_model(x.numpy().reshape([1, 28, 28, 1]))
    print(y, tf.math.sigmoid(pred))
    print(pred)
    i += 1
    print(i, end="\r")
    if i > 100:
        break
    if pred > 0.9 and y == wünscheswertes_char:
        stimt.append((x, y))

tf.Tensor([1], shape=(1,), dtype=int64) tf.Tensor([[0.00023868]], shape=(1, 1), dtype=float32)
tf.Tensor([[-8.340165]], shape=(1, 1), dtype=float32)
tf.Tensor([1], shape=(1,), dtype=int64) tf.Tensor([[0.00029057]], shape=(1, 1), dtype=float32)
tf.Tensor([[-8.14338]], shape=(1, 1), dtype=float32)
tf.Tensor([1], shape=(1,), dtype=int64) tf.Tensor([[0.00012853]], shape=(1, 1), dtype=float32)
tf.Tensor([[-8.959198]], shape=(1, 1), dtype=float32)
tf.Tensor([1], shape=(1,), dtype=int64) tf.Tensor([[0.00023313]], shape=(1, 1), dtype=float32)
tf.Tensor([[-8.363695]], shape=(1, 1), dtype=float32)
tf.Tensor([1], shape=(1,), dtype=int64) tf.Tensor([[0.00082119]], shape=(1, 1), dtype=float32)
tf.Tensor([[-7.1039333]], shape=(1, 1), dtype=float32)
tf.Tensor([1], shape=(1,), dtype=int64) tf.Tensor([[0.00018286]], shape=(1, 1), dtype=float32)
tf.Tensor([[-8.606631]], shape=(1, 1), dtype=float32)
tf.Tensor([1], shape=(1,), dtype=int64) tf.Tensor([[0.00022896]], shape=(1, 1), dtype=float32)
tf.Tensor([

In [93]:
for x, y in nicht_potenzielle_chars:
    pred = clone_cnn_model(x.numpy().reshape([1, 28, 28, 1]))
    print(y, tf.math.sigmoid(pred))
    print(pred)
    i += 1
    print(i, end="\r")
    if i > 1000:
        break
    if pred > 0.9 and y == wünscheswertes_char:
        stimt.append((x, y))

tf.Tensor([9], shape=(1,), dtype=int64) tf.Tensor([[0.00010807]], shape=(1, 1), dtype=float32)
tf.Tensor([[-9.132627]], shape=(1, 1), dtype=float32)
tf.Tensor([7], shape=(1,), dtype=int64) tf.Tensor([[0.00015514]], shape=(1, 1), dtype=float32)
tf.Tensor([[-8.771055]], shape=(1, 1), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64) tf.Tensor([[0.00017201]], shape=(1, 1), dtype=float32)
tf.Tensor([[-8.667803]], shape=(1, 1), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64) tf.Tensor([[0.00015205]], shape=(1, 1), dtype=float32)
tf.Tensor([[-8.79112]], shape=(1, 1), dtype=float32)
tf.Tensor([8], shape=(1,), dtype=int64) tf.Tensor([[0.00029005]], shape=(1, 1), dtype=float32)
tf.Tensor([[-8.145174]], shape=(1, 1), dtype=float32)
tf.Tensor([8], shape=(1,), dtype=int64) tf.Tensor([[0.00010798]], shape=(1, 1), dtype=float32)
tf.Tensor([[-9.133481]], shape=(1, 1), dtype=float32)
tf.Tensor([6], shape=(1,), dtype=int64) tf.Tensor([[9.881671e-05]], shape=(1, 1), dtype=float32)
tf.Tensor(

In [70]:
clone_cnn_model(x.numpy().reshape([1, 28, 28])), y

(<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[35.891907]], dtype=float32)>,
 <tf.Tensor: shape=(), dtype=int64, numpy=17>)

In [62]:
for x, y in kmnist_ts:
    print(x.shape, y.shape)

(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1) ()
(28, 28, 1

KeyboardInterrupt: 

In [53]:
!python --version

Python 3.10.4
